In [1]:
import os
import pandas as pd
import random


In [2]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2 as cv2

In [3]:
Directory = 'C:\\Users\\Arnav Anand\\Documents\\AI_ML_project\\alphabet\\Alpha'
categories = ['anew','bnew']

count =0;

data = []

target = []

for cat in categories:
    folder = os.path.join(Directory,cat)
    label = categories.index(cat)
    for img in os.listdir(folder):
        img_path = os.path.join(folder,img)
        img_arr = cv2.imread(img_path)
        gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        resize_image = cv2.resize(gray,(150,150))
        data.append(resize_image)
        target.append(count)
        
    count+=1

In [4]:
len(data)

597

In [5]:
count

2

In [6]:
data_normalized = np.array(data)/255.0
data_normalized_reshaped = data_normalized.reshape(597,150,150,1)

In [7]:
data_normalized_reshaped

array([[[[0.05098039],
         [0.05098039],
         [0.05490196],
         ...,
         [0.16862745],
         [0.16862745],
         [0.16862745]],

        [[0.05490196],
         [0.05490196],
         [0.05490196],
         ...,
         [0.17254902],
         [0.17254902],
         [0.17254902]],

        [[0.05490196],
         [0.05490196],
         [0.05490196],
         ...,
         [0.17254902],
         [0.17254902],
         [0.17254902]],

        ...,

        [[0.17647059],
         [0.16862745],
         [0.16078431],
         ...,
         [0.49019608],
         [0.49019608],
         [0.48627451]],

        [[0.17647059],
         [0.17254902],
         [0.16470588],
         ...,
         [0.48627451],
         [0.48627451],
         [0.48627451]],

        [[0.17647059],
         [0.17254902],
         [0.16862745],
         ...,
         [0.48235294],
         [0.47843137],
         [0.47843137]]],


       [[[0.56470588],
         [0.60784314],
         [0.69

In [8]:
import tensorflow
from keras.utils import np_utils
target = np_utils.to_categorical(target)
target

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(data_normalized_reshaped, target)


In [11]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


((447, 150, 150, 1), (150, 150, 150, 1), (447, 2), (150, 2))

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint

In [13]:
model = Sequential()
model.add(Conv2D(300, (3,3), input_shape=(150,  150, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(100, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
cp = ModelCheckpoint('best_one', verbose=0, save_best_only=True)
model.fit(X_train, Y_train, epochs = 5, callbacks=[cp], validation_split=0.2)


Epoch 1/5
12/12 [==============================] - 164s 13s/step - loss: 0.8011 - accuracy: 0.4930 - val_loss: 0.7121 - val_accuracy: 0.4333
INFO:tensorflow:Assets written to: best_one\assets
Epoch 2/5
12/12 [==============================] - 173s 14s/step - loss: 0.5757 - accuracy: 0.7339 - val_loss: 0.5417 - val_accuracy: 0.7889
INFO:tensorflow:Assets written to: best_one\assets
Epoch 3/5
12/12 [==============================] - 147s 12s/step - loss: 0.5716 - accuracy: 0.6779 - val_loss: 0.3775 - val_accuracy: 0.8444
INFO:tensorflow:Assets written to: best_one\assets
Epoch 4/5
12/12 [==============================] - 171s 14s/step - loss: 0.3856 - accuracy: 0.8403 - val_loss: 0.3129 - val_accuracy: 0.8889
INFO:tensorflow:Assets written to: best_one\assets
Epoch 5/5
12/12 [==============================] - 143s 12s/step - loss: 0.2661 - accuracy: 0.8739 - val_loss: 0.2879 - val_accuracy: 0.8778
INFO:tensorflow:Assets written to: best_one\assets


In [15]:
from keras.models import load_model
model = load_model('best_one')

In [16]:
os.chdir('C:\\Users\\Arnav Anand\\Documents\\AI_ML_project\\haarcascades')
gesture_detect=cv2.CascadeClassifier('hand.xml')

In [22]:
source = cv2.VideoCapture(0)

while 1:

    _,image =source.read(0)

    #print ("Captured image")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detected_gesture = gesture_detect.detectMultiScale(gray, 1.2, 9)

    #print("Detected faces", Len(detected faces))

    if len(detected_gesture) > 1:

        continue

    for each_gesture in detected_gesture: #Each bbox is a rectangle representing

        # the bounding box around the detected object

        x, y, w, h = each_gesture 
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 3)

        #Cropped object

        crop= gray[y:y+h, x:x+w]

        cropped_image = cv2.resize(crop, (150, 150)) 
        normalized_image = cropped_image/255

        reshaped_gesture = np.reshape(normalized_image,(1,150,150,1))
        result = model.predict(reshaped_gesture)[0]

        if result[0]> result[1]:

            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2) 
            cv2.putText(image, "letter A", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if result[1] > result[0]:

            cv2.rectangle(image, (x, y), (x+w, y + h), (0, 0, 255), 2) 
            cv2.putText(image, "letter B", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("CV_Project", image)

    key= cv2.waitKey(1)

    if key == 27: 
        break

cv2.destroyAllWindows()

source.release()